# **STATIC CLIENT**

You've just finished eavesdropping on a conversation between Alice and Bob. Now you have a chance to talk to Bob. What are you going to say?

Connect at socket.cryptohack.org 13373

In [ ]:
!pip install telnetlib

ERROR: Could not find a version that satisfies the requirement telnetlib (from versions: none)
ERROR: No matching distribution found for telnetlib


In [ ]:
import telnetlib
import json
from modexp import modexp

HOST = "socket.cryptohack.org"
PORT = 13373

tn = telnetlib.Telnet(HOST, PORT)

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline().decode()
    line = line[line.find("{"):]
    return json.loads(line)

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

def getInitMessage():
  d = json_recv()
  p = int(d["p"], 16)
  g = int(d["g"], 16)
  A = int(d["A"], 16)
  return (p,g,A)

def createSharedSecret(pk, sk, p):
  return modexp(pk, sk, p)

def makePK(sk, g, p):
  return modexp(g, sk, p)

def sendToBob(p,g,A):
  d = {"p":p, "g":g, "A":A}
  json_send(d)

def getFromB():
  d = json_recv()
  B = int(d["B"], 16)
  return B

def sendToAlice(B):
  d = {"B":B}
  json_send(d)

def getIvAndFlag():
  d = json_recv()
  iv = d["iv"]
  encrypted_flag = d["encrypted"]
  return (iv, encrypted_flag)

(p,g,A) = getInitMessage() # From A
B = getFromB() # from B
(iv, enc_flag) = getIvAndFlag() # From A

sendToBob(hex(p), hex(A), "0x01") # To Bob
sharedK = getFromB() # from B

from decrypt import decrypt_flag
flag = decrypt_flag(sharedK, iv, enc_flag)
print(flag)

ModuleNotFoundError: No module named 'modexp'

In [ ]:
import telnetlib
import json

def modexp(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base ** 2) % modulus
    return result

def communicate_with_bob():
    HOST = "socket.cryptohack.org"
    PORT = 13373

    tn = telnetlib.Telnet(HOST, PORT)

    def readline():
        return tn.read_until(b"\n")

    def json_recv():
        line = readline().decode()
        line = line[line.find("{"):]
        return json.loads(line)

    def json_send(hsh):
        request = json.dumps(hsh).encode()
        tn.write(request)

    def get_init_message():
        d = json_recv()
        p = int(d["p"], 16)
        g = int(d["g"], 16)
        A = int(d["A"], 16)
        return p, g, A

    def create_shared_secret(pk, sk, p):
        return modexp(pk, sk, p)

    def make_pk(sk, g, p):
        return modexp(g, sk, p)

    def send_to_bob(p, g, A):
        d = {"p": p, "g": g, "A": A}
        json_send(d)

    def get_from_b():
        d = json_recv()
        B = int(d["B"], 16)
        return B

    def send_to_alice(B):
        d = {"B": B}
        json_send(d)

    def get_iv_and_flag():
        d = json_recv()
        iv = d["iv"]
        encrypted_flag = d["encrypted"]
        return iv, encrypted_flag

    # From A
    p, g, A = get_init_message()
    # To Bob
    send_to_bob(hex(p), hex(A), "0x01")
    # From B
    B = get_from_b()
    # To A
    send_to_alice(B)
    # From A
    iv, enc_flag = get_iv_and_flag()

    # Now you can communicate with Bob or perform any necessary actions

    tn.close()

# Call the function to start communication with Bob
communicate_with_bob()


In [ ]:
import telnetlib
import json

def modexp(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base ** 2) % modulus
    return result

def decrypt_flag(key, iv, encrypted_flag):
    key_bytes = key.to_bytes((key.bit_length() + 7) // 8, 'big')  # Convert key to bytes
    encrypted_bytes = bytes.fromhex(encrypted_flag)  # Convert the hexadecimal string to bytes
    decrypted_flag = bytearray()

    for i in range(len(encrypted_bytes)):
        decrypted_byte = encrypted_bytes[i] ^ key_bytes[i % len(key_bytes)]
        decrypted_flag.append(decrypted_byte)

    return decrypted_flag.hex()



def communicate_with_bob():
    HOST = "socket.cryptohack.org"
    PORT = 13373

    tn = telnetlib.Telnet(HOST, PORT)

    def readline():
        return tn.read_until(b"\n")

    def json_recv():
        line = readline().decode()
        line = line[line.find("{"):]
        return json.loads(line)

    def json_send(hsh):
        request = json.dumps(hsh).encode()
        tn.write(request)

    def get_init_message():
        d = json_recv()
        p = int(d["p"], 16)
        g = int(d["g"], 16)
        A = int(d["A"], 16)
        return p, g, A

    def create_shared_secret(pk, sk, p):
        return modexp(pk, sk, p)

    def make_pk(sk, g, p):
        return modexp(g, sk, p)

    def send_to_bob(p, g, A):
        d = {"p": p, "g": g, "A": A}
        json_send(d)

    def get_from_b():
        d = json_recv()
        B = int(d["B"], 16)
        return B

    def send_to_alice(B):
        d = {"B": B}
        json_send(d)

    def get_iv_and_flag():
        d = json_recv()
        iv = d["iv"]
        encrypted_flag = d["encrypted"]
        return iv, encrypted_flag

    # From A
    p, g, A = get_init_message()

    # To Bob
    send_to_bob(hex(p), hex(A), "0x01")

    # From B
    B = get_from_b()

    # Shared key calculation
    sharedK = create_shared_secret(B, A, p)

    # To A
    send_to_alice(B)

    # From A
    iv, enc_flag = get_iv_and_flag()

    # Decrypting the flag
    flag = decrypt_flag(sharedK, iv, enc_flag)
    return flag

# Call the function to start communication with Bob
flag = communicate_with_bob()

# Print the obtained flag as hexadecimal
print("Flag:", flag)

# Alternatively, print the obtained flag as base64
import base64
print("Flag (Base64):", base64.b64encode(bytes.fromhex(flag)).decode('utf-8'))




Flag: 074aed334b23c000c9710f0beee6f53327a3e88d4007445acb89552a4e5b672b
Flag (Base64): B0rtM0sjwADJcQ8L7ub1Myej6I1AB0Ray4lVKk5bZys=


In [ ]:
pip install ptrlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00


In [ ]:
import hashlib
import json

import ptrlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad


sock = ptrlib.Socket("socket.cryptohack.org", 13373)

hex2int = lambda s: int(s[2:], 16)
parse_ct = lambda msg: tuple(bytes.fromhex(msg[x]) for x in ('iv', 'encrypted'))
intercept_alice = lambda: json.loads(sock.recvlineafter('Intercepted from Alice: '))
intercept_bob = lambda: json.loads(sock.recvlineafter('Intercepted from Bob: '))
recv_bob = lambda: json.loads(sock.recvlineafter('Bob says to you: '))
send_bob = lambda payload: sock.sendlineafter('Bob connects to you, send him some parameters: ', json.dumps(payload))

def decrypt_msg(shared_secret, iv, ct):
    key = hashlib.sha1(str(shared_secret).encode('ascii')).digest()[:16]

    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), 16)

    return pt

# ---

msg = intercept_alice()
p, g, A = (hex2int(msg[x]) for x in 'pgA')

msg = intercept_bob()
B = hex2int(msg['B'])

msg = intercept_alice()
iv, ct = parse_ct(msg)

send_bob({ 'p': hex(p), 'g': hex(A), 'A': hex(0) })

msg = recv_bob()
B2 = hex2int(msg['B'])

msg = recv_bob()
iv2, ct2 = parse_ct(msg)

# ---

pt2 = decrypt_msg(0, iv2, ct2)
print(pt2)  # "Hey, what's up. I got bored generating random numbers did you see?"

# Bob is reusing his private key in different conversations -- not good (for him).
# B2 is now A**b, the shared secret key in the first conversation.
pt = decrypt_msg(B2, iv, ct)
print(pt)  # crypto{n07_3ph3m3r4l_3n0u6h}

[+] __init__: Successfully connected to socket.cryptohack.org:13373
INFO:ptrlib.connection.sock:Successfully connected to socket.cryptohack.org:13373


b"Hey, what's up. I got bored generating random numbers did you see?"
b'crypto{n07_3ph3m3r4l_3n0u6h}'


# **GROUP THEORY**

# Additive

Alice and Bob decided to do their DHKE in an additive group rather than a multiplicative group. What could go wrong?

Use the script from "Diffie-Hellman Starter 5" to decrypt the flag once you've recovered the shared secret.

Connect at socket.cryptohack.org 13380

In [ ]:
!pip install pwntools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 14.4 MB/s eta 0:00:00
  Created wheel for in

In [ ]:
#!/usr/bin/env python3
from telnetlib import Telnet
import json
from Crypto.Util.number import *
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib

def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))

def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    try:
        return json.loads(line.decode())
    except:
        print(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

with Telnet("socket.cryptohack.org", 13380) as tn:
    tn.read_until(b"Intercepted from Alice: ")
    data = json_recv()
    p = int(data["p"], 16)
    g = int(data["g"], 16)
    A = int(data["A"], 16)

    tn.read_until(b"Intercepted from Bob: ")
    data = json_recv()
    B = int(data["B"], 16)

    tn.read_until(b"Intercepted from Alice: ")
    data = json_recv()
    iv = data["iv"]
    encrypted = data["encrypted"]

# Alice calculates A = g + g + ... + g (mod p) = a * g (mod p)
#                      ^^^^^^^^^^^^^^^ a times
# so a = A / g
a = (A + (p if A % 2 else 0)) // g
assert (a * g) % p == A

# Bob calculates B = g + g + ... + g (mod p) = b * g (mod p)
#                    ^^^^^^^^^^^^^^^ b times
# so b = B / g
b = (B + (p if B % 2 else 0)) // g
assert (b * g) % p == B

K = (b * A) % p
print(decrypt_flag(K, iv, encrypted))

crypto{cycl1c_6r0up_und3r_4dd1710n?}


# **STATIC CLIENT 2**

Bob got a bit more careful with the way he verifies parameters. He's still insisting on using the p and g values provided by his partner. Wonder if he missed anything?

Connect at socket.cryptohack.org 13378


In [ ]:
!pip install gmpy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.6 MB/s eta 0:00:00


In [ ]:
import gmpy2
import json
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import telnetlib
import random
#from sympy.ntheory.modular import crt
from tqdm import tqdm



def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))

def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytearray.fromhex(ciphertext)
    iv = bytearray.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)
    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')

def product(l):
    p = 1
    for x in l:
        p *= x
    return p

def CRT(a, n):
    l = len(a)
    N = product(n)
    ni = [N // n[j] for j in range(l)]
    inv_ni = [gmpy2.invert(ni[j], n[j]) for j in range(l)]
    x = [a[j] * ni[j] * inv_ni[j] for j in range(l)]
    return sum(x) % N


original_P=0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff

HOST = "socket.cryptohack.org"
PORT = 13378

# generate the first 1,000 prime numbers
primes = [2]
for i in range(1000):
    primes.append(int(gmpy2.next_prime(primes[-1])))
primes=primes[100:]  #keep just big primes

# generate a weak prime (P) such that P>original_P
while True:
    N = 2
    factors=[]
    while (N<original_P):
        prime=random.choice(primes)
        if prime not in factors:
            factors.append(prime)
            N*=prime
    if gmpy2.is_prime(N+1):
        break
P=N+1

# Find remainders for chineses ramaindeder theory
remainders=[]
for prime in tqdm(factors):
    generator_candidate = random.randrange(2, P - 1)
    candidate = pow(generator_candidate, (P - 1) // prime, P)
    if candidate != (P - 1) and candidate != 1:
        tn = telnetlib.Telnet(HOST, PORT)
        for _ in range(3):
            line=tn.read_until(b"\n")
        tn.read_until(b': ')
        d = {"p": hex(P), "g": "0x02", "A":hex(candidate) }
        m = json.dumps(d).encode()
        tn.write(m)
        r=tn.read_until(b"\n")[17:]
        B = int(json.loads(r)['B'],16)
        r=tn.read_until(b"\n")[17:]
        iv=json.loads(r)['iv']
        encrypted=json.loads(r)['encrypted']
        possible_shared = set()
        for i in range(1,prime):
            possible_shared=pow(candidate, i, P)
            try:
                pt = decrypt_flag(possible_shared, iv, encrypted)
                if "My new Diffie-Hellman" in pt:
                    remainders.append(i)
            except:
                pass
b = CRT(remainders,factors)   # get private key of Bob (b)
tn = telnetlib.Telnet(HOST, PORT)
A=json.loads(tn.read_until(b"\n")[24:])['A']
B=tn.read_until(b"\n")
r=tn.read_until(b"\n")[24:]
iv=json.loads(r)['iv']
encrypted=json.loads(r)['encrypted']
secret_shared=pow(int(A,16),b,original_P)
print(decrypt_flag(secret_shared, iv, encrypted))

100%|██████████| 131/131 [03:16<00:00,  1.50s/it]


crypto{uns4f3_pr1m3_sm4ll_oRd3r}


# **MISC**

# **Script Kiddie**

Found this cool script on Github and I've been using it to keep my secrets from anyone listening in on the school wifi!

Challenge files:
  - script.py
  - output.txt

In [ ]:
from Crypto.Cipher import AES
import hashlib

def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def pkcs7_unpad(message, block_size=16):
    if len(message) == 0:
        raise Exception("The input data must contain at least one byte")
    if not is_pkcs7_padded(message):
        return message
    padding_len = message[-1]
    return message[:-padding_len]


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)
    return pkcs7_unpad(plaintext).decode('ascii')


p= 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g= 2
A= 539556019868756019035615487062583764545019803793635712947528463889304486869497162061335997527971977050049337464152478479265992127749780103259420400564906895897077512359628760656227084039215210033374611483959802841868892445902197049235745933150328311259162433075155095844532813412268773066318780724878693701177217733659861396010057464019948199892231790191103752209797118863201066964703008895947360077614198735382678809731252084194135812256359294228383696551949882
B= 652888676809466256406904653886313023288609075262748718135045355786028783611182379919130347165201199876762400523413029908630805888567578414109983228590188758171259420566830374793540891937904402387134765200478072915215871011267065310188328883039327167068295517693269989835771255162641401501080811953709743259493453369152994501213224841052509818015422338794357540968552645357127943400146625902468838113443484208599332251406190345653880206706388377388164982846343351
iv= 'c044059ae57b61821a9090fbdefc63c5'
encrypted_flag= 'f60522a95bde87a9ff00dc2c3d99177019f625f3364188c1058183004506bf96541cf241dad1c0e92535564e537322d7'

# notation indicates XOR function used not pow
b = B ^ g % p

shared = b ^ A % p
flag = decrypt_flag(shared, iv, encrypted_flag)
print(f'[+] Flag recovered: {flag}')

[+] Flag recovered: crypto{b3_c4r3ful_w1th_y0ur_n0tati0n}


# **The Matrix**


I must get out of here. I must get free, and in this mind is the key, my key!

Challenge files:
  - the_matrix.sage
  - flag.enc


Challenge contributed by jschnei

In [ ]:
pip install pycryptodome sympy

In [ ]:
from Crypto.Util.number import long_to_bytes
from sympy import GF, Matrix
from sympy.matrices.groups import GL

P = 2
N = 50
E = 31337

FLAG = b'crypto{??????????????????????????}'

def bin_to_bytes(s):
    all_bytes = [s[i:i+8] for i in range(0, len(s), 8)]
    return b''.join(long_to_bytes(int(byte, 2)) for byte in all_bytes)

# Placeholder for the missing function
def load_matrix(file_name):
    # Your implementation to load the matrix from the file goes here
    pass

# https://en.wikipedia.org/wiki/General_linear_group#Over_finite_fields
def find_GL_order():
    order = 1
    k = 0
    for k in range(N):
        order *= (P**N - P**k)

    assert order == GL(N, GF(P)).order()

    return order

C = load_matrix('flag.enc')

GLorder = find_GL_order()

D = pow(E, -1, GLorder)   # either GLorder or C.multiplicative_order()

M = C**D

cols = M.columns()[:(len(FLAG)*8//50)+1]

bin_flag = ''.join([str(bit) for col in cols for bit in col])

print(bin_to_bytes(bin_flag))


ModuleNotFoundError: No module named 'sympy.matrices.groups'

# **The Matrix Reloaded**

It's happening exactly as before... Well, not exactly.

Challenge files:
  - matrix_reloaded.zip


Challenge contributed by jschnei

In [ ]:
from Crypto.Util.Padding import unpad
from Crypto.Cipher import AES
from hashlib import sha256
def decrypt_flag(shared_secret, iv, ciphertext):
    key = sha256(str(shared_secret).encode()).digest()[:128]
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    plaintext = AES.new(key, AES.MODE_CBC, iv).decrypt(ciphertext)
    return unpad(plaintext, 16).decode()

e = 5959805911241109643914928800631944794321671043586961836890946136294554770507810148857251869110638484873235200204605081157845088692257708370810040562721345
iv = "334b1ceb2ce0d1bef2af9937cf82aad6"
ciphertext = "543e29415bdb1f694a705b2532a5beb7ebd7009591503ef3c4fbcebf9e62fe91307e5d98efcd49f9f3b1985956cafc89"

print(decrypt_flag(e, iv, ciphertext))

crypto{the_oracle_told_me_about_you_91e019ff}


# **The Matrix Revolutions**

Everything that has a beginning has an end, Neo.

Challenge files:
  - matrix_revolutions.zip


In [ ]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto.Util.Padding import unpad
import numpy as np
from sympy import Matrix
from math import crt

P = 2
N = 150

def load_matrix(fname):
    data = open(fname, 'r').read().strip()
    rows = [list(map(int, row)) for row in data.splitlines()]
    return Matrix(rows)

G = load_matrix('generator.txt')
A_pub = load_matrix('alice.pub')
B_pub = load_matrix('bob.pub')

f = G.charpoly()

X = []
M = []
for g, e in f.factor():
    assert e == 1
    K = GF(2**g.degree(), 'x', modulus=g, impl='pari_ffelt')
    a = g.roots(K)[0][0]
    w = (G - a * Matrix.eye(N)).nullspace()[0]
    V = [Matrix([0] * i + [1] + [0] * (N - 1 - i)).transpose() for i in range(150)]
    P = Matrix.hstack(w, *V[:-1]).transpose()
    assert P.rank() == N
    J_ = ~P * A_pub * P
    X.append(int(J_[0, 0].log(a)))
    M.append(K.multiplicative_generator().multiplicative_order())

# Función crt para calcular el teorema del resto chino
def chinese_remainder_theorem(a, m):
    result, _ = crt(X, M)
    return result

A_priv = chinese_remainder_theorem(X, M)
shared_secret = B_pub**A_priv

iv = '43f14157442d75142d0d4993e99a9582'
ciphertext = '22abc3b347ffef55ec82488e5b4a338da5af7ef1918ac46f95029a4d94ace4cb2700fa9aeb31e6a4facee2601e99dabd6f9a81494c55f011e9227c9a6ae8d802'
KEY_LENGTH = 128

def derive_aes_key(M):
    mat_str = ''.join(str(x) for row in M for x in row)
    return SHA256.new(data=mat_str.encode()).digest()[:KEY_LENGTH]

key = derive_aes_key(shared_secret)
cipher = AES.new(key, AES.MODE_CBC, bytes.fromhex(iv))
flag = unpad(cipher.decrypt(bytes.fromhex(ciphertext)), 16).decode()

print(flag)


ImportError: cannot import name 'crt' from 'math' (unknown location)